<a href="https://colab.research.google.com/github/patrickdevv/FIAP_Fase03_Desafio/blob/main/Desafio03_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9m9t_2j9/unsloth_815a77c088b14dde9ef7cd439b4d97ad
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9m9t_2j9/unsloth_815a77c088b14dde9ef7cd439b4d97ad
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 9.6 MB/s eta 0:00:00
   ━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.12.1
    Uninstalling trl-0.12.1:
      Successfully uninstalled trl-0.12.1


#Tratamento do arquivo

In [3]:
import pandas as pd
import json

## Devido aos erros no arquivo, vamos processar JSON Linha a Linha

In [5]:

# Caminho do arquivo JSON
caminho_arquivo = '/content/drive/MyDrive/Colab Notebooks/desafio03/trn.json'

# Abrir o arquivo e ler linha por linha
with open(caminho_arquivo, 'r') as f:
    data = [json.loads(line) for line in f]

# Converter em DataFrame
df = pd.DataFrame(data)

# Visualizar o DataFrame
print(df.head())

          uid                                              title  \
0  0000031909                        Girls Ballet Tutu Neon Pink   
1  0000032034                           Adult Ballet Tutu Yellow   
2  0000913154  The Way Things Work: An Illustrated Encycloped...   
3  0001360000                                      Mog's Kittens   
4  0001381245                              Misty of Chincoteague   

                                             content  \
0  High quality 3 layer ballet tutu. 12 inches in...   
1                                                      
2                                                      
3  Judith Kerr&#8217;s best&#8211;selling adventu...   
4                                                      

                                          target_ind  \
0  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...   
1  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37,...   
2                [116, 117, 118, 119, 120, 121, 122]   
3                          [14

## Filtrar o arquivo deixando somente as colunas Title e Content

In [6]:
# Filtrar apenas as colunas necessárias
df_filtrado = df[['title', 'content']]

# Nome do novo arquivo
nome_arquivo = '/content/drive/MyDrive/Colab Notebooks/desafio03/trn_filtrado.json'

df_filtrado.to_json(nome_arquivo, orient='records', lines=True)

print(f"Arquivo JSON filtrado salvo em: {nome_arquivo}")


Arquivo JSON filtrado salvo em: /content/drive/MyDrive/Colab Notebooks/desafio03/trn_filtrado.json


In [ ]:
# Verificar informações gerais do DataFrame
df_filtrado.info()

In [ ]:
# Visualizar o DataFrame
print(df_filtrado.head())
print(df_filtrado.count())

In [7]:
# Remover registros onde 'content' está vazio ou contém apenas espaços
df_filtrado_limpo = df_filtrado[df_filtrado['content'].str.strip() != '']

# Substituir " por ' na coluna 'content'
df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('"', "'"))

# Substituir todas as ocorrências de \" por ' em 'content'
df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('\\"', "'"))

# Verificar o resultado
print(df_filtrado_limpo.head())

<ipython-input-7-53f250acb3d7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('"', "'"))


                          title  \
0   Girls Ballet Tutu Neon Pink   
3                 Mog's Kittens   
7   Girls Ballet Tutu Neon Blue   
12                  The Prophet   
13    Rightly Dividing the Word   

                                              content  
0   High quality 3 layer ballet tutu. 12 inches in...  
3   Judith Kerr&#8217;s best&#8211;selling adventu...  
7   Dance tutu for girls ages 2-8 years. Perfect f...  
12  In a distant, timeless place, a mysterious pro...  
13         --This text refers to thePaperbackedition.  


<ipython-input-7-53f250acb3d7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('\\"', "'"))


In [ ]:
print(df_filtrado_limpo.count())

## Criar json para utilização em treinamento do LLM

In [8]:
import json

# Parte inicial e final para o prompt
parte_inicial_question = "[|Question|] What is the description for the product '"
parte_final_question = "'? [|eQuestion|]"
parte_inicial_answer = "[|Answer|] '"
parte_final_answer = "' [|eAnswer|]"

# Construir a estrutura desejada
novo_json = [
    {
        "prompt": f"{parte_inicial_question}{row['title']}{parte_final_question}",
        "completion": f"{parte_inicial_answer}{row['content']}{parte_final_answer}"
    }
    for _, row in df_filtrado_limpo.iterrows()
]

# Salvar o novo JSON estruturado
caminho_saida = 'trn_test.json'
with open(caminho_saida, 'w') as f:
    json.dump(novo_json, f, ensure_ascii=False, indent=4)

print(f"Novo JSON formatado salvo em: {caminho_saida}")


Novo JSON formatado salvo em: trn_test.json


### Abre arquivo preparado para teste

In [ ]:
# Caminho do arquivo JSON
caminho_arquivo_test = '/content/drive/MyDrive/Colab Notebooks/desafio03/trn_test.json'

# Abrir o arquivo e ler linha por linha
with open(caminho_arquivo_test, 'r') as f:
    data = [json.loads(line) for line in f]

# Converter em DataFrame
df_preparado_test = pd.DataFrame(data)

JSONDecodeError: Expecting value: line 2 column 1 (char 2)

#Prepara início do treinamento

In [9]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/desafio03/trn_test.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/desafio03/trn_test_output.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:
novo_json[0]

{'prompt': "[|Question|] What is the description for the product 'Girls Ballet Tutu Neon Pink'? [|eQuestion|]",
 'completion': "[|Answer|] 'High quality 3 layer ballet tutu. 12 inches in length' [|eAnswer|]"}

## Não precisa executar a busca pelo arquivo se estiver executando desde o início

In [ ]:
with open(DATA_PATH, 'r', encoding='utf-8') as file:
        data = json.load(file)
        print('data: ',data)

NameError: name 'DATA_PATH' is not defined

In [15]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        question_start = full_text.find("[|Question|]") + len("[|Question|]")
        question_end = full_text.find("[|eQuestion|]")
        answer_start = full_text.find("[|Answer|]") + len("[|Answer|]")
        answer_end = full_text.find("[|eAnswer|]")

        # Extrair partes relevantes do texto
        instruction = full_text.split('\n')[0]  # Primeira linha como instrução
        input_text = full_text[question_start:question_end].strip()
        response = full_text[answer_start:answer_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Iterar diretamente sobre a lista de dados
    for item in data:
        prompt = item['prompt']  # Extrair o campo 'prompt' de cada item
        completion = item['completion']  # Extrair o campo 'completion' de cada item

        # Separar os textos em partes relevantes
        instruction, input_text, response = separate_text(prompt + "\n" + completion)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/desafio03/formatted_dataset.json"
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")


In [16]:


format_dataset_into_model_input(novo_json)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Dataset salvo em /content/drive/MyDrive/Colab Notebooks/desafio03/formatted_dataset.json
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [19]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/desafio03/formatted_dataset.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1498718 [00:00<?, ? examples/s]

In [20]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1498718 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,498,718 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.647200
2,2.662700
3,2.687900
4,2.584500
5,2.220900
6,2.276600
7,2.046000
8,1.822700
9,1.856200
10,1.546600


In [22]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer this question",
        " What is the description for the product 'Girls Ballet Tutu Neon Pink", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer this question\n\n### Input:\n What is the description for the product 'Girls Ballet Tutu Neon Pink\n\n### Response:\n'This tutu is made of 100% polyester and is made in the USA. This tutu is 100% machine washable. This tutu has a satin waistband with a drawstring. This tutu is available in size 4-6T. This tutu has a 12 inch drop"]

In [23]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer this question",
        " What is the description for the product 'Girls Ballet Tutu Neon Pink", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer this question

### Input:
 What is the description for the product 'Girls Ballet Tutu Neon Pink

### Response:
 'Neon pink ballet tutu with a large pink rose on the front. It is made of tulle and has a satin elastic waistband. The tutu is a size 4T. It is made by a professional ballet company in New York City. The tutu is brand new and never been worn.'<|end_of_text|>


In [24]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model")


('/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model/tokenizer.json')